In [ ]:
%pylab inline
import matplotlib
import matplotlib.pyplot as plt
from netCDF4 import Dataset
from wrf import getvar, interplevel, destagger, smooth2d
import numpy as np
from matplotlib.cm import get_cmap
from matplotlib.ticker import MultipleLocator
import matplotlib.transforms as mtransforms

import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
from functions import plot, calc

nc1 = Dataset('/p/work1/lloveras/adj_4km/wrfin_ctl')

In [ ]:
u = np.asarray(getvar(nc1,'ua'))
th = np.asarray(getvar(nc1,'th'))
qv = np.asarray(getvar(nc1,'QVAPOR'))
pvo = np.asarray(getvar(nc1,'pvo'))
p = np.asarray(getvar(nc1,'pressure'))
z = np.asarray(getvar(nc1,'z'))/1000.

In [ ]:
z1 = 0
z2 = 100
dz = 0.2
y1 = 0
y2 = 1800
x1 = 0
x2 = 2000
dx = 4
plev = 250
x = 0

zgrid = np.arange(z1,z2)
ygrid = np.arange(y1*dx,y2*dx,dx)
y,zmesh = np.meshgrid(ygrid,zgrid)

u_cross = u[z1:z2,y1:y2,x]
th_cross = th[z1:z2,y1:y2,x]
qv_cross = qv[z1:z2,y1:y2,x]
pv_cross = pvo[z1:z2,y1:y2,x]
z_cross = z[z1:z2,y1:y2,x]

t_plan = th[0,y1:y2,x1:x2]
zlev = interplevel(z*100,p,plev)

yax1 = np.arange(y1*dx,(y2+1)*dx,1000).astype(int)
yax2 = (yax1 - y1*dx)/float(dx)
yax2 = yax2.astype(int)
xax1 = np.arange(x1*dx,(x2+1)*dx,1000).astype(int)
xax2 = (xax1 - x1*dx)/float(dx)
xax2 = xax2.astype(int)


In [ ]:
fig, axd = plt.subplot_mosaic([['left','right'],['cbar1','cbar2']],
                              constrained_layout=True, figsize=(14.0,7.0), 
                              gridspec_kw={'width_ratios':[1,1],'height_ratios':[1,0.05]})#,dpi=100)

trans = mtransforms.ScaledTranslation(1/25, -1/25, fig.dpi_scale_trans)

im1 = axd['left'].contourf(y,z_cross,qv_cross*1000,levels=np.arange(2,14,2),extend='max',cmap=get_cmap('GnBu'))
cbar1 = fig.colorbar(im1, orientation="horizontal", cax=axd['cbar1'])
cbar1.ax.tick_params(labelsize=20, pad=6)
cs1 = axd['left'].contour(y,z_cross,th_cross,levels=np.arange(200,560,10),colors='grey',linewidths=2,alpha=0.9)
axd['left'].clabel(cs1,fmt='%1.0f',inline=1,levels=[270,300,330],fontsize=12,colors='k')
cs2 = axd['left'].contour(y,z_cross,u_cross,levels=np.arange(5,105,5),colors='red',linewidths=2.0)
axd['left'].clabel(cs2,fmt='%1.0f',inline=1,fontsize=12)
cs3 = axd['left'].contour(y,z_cross,pv_cross,levels=[1.5],colors='k',linewidths=3.5)
axd['left'].tick_params(axis='both', labelsize=20, pad=8,length=8,which='both')
axd['left'].set_xlabel('South-North (km)',fontsize=20,labelpad=6)
axd['left'].set_ylabel('Height (km)',labelpad=6,fontsize=20)
axd['left'].set_ylim((0,20))
axd['left'].yaxis.set_major_locator(MultipleLocator(4))
axd['left'].yaxis.set_major_formatter(FormatStrFormatter('%d'))
axd['left'].yaxis.set_minor_locator(MultipleLocator(2))
axd['left'].xaxis.set_major_locator(MultipleLocator(2000))
axd['left'].xaxis.set_major_formatter(FormatStrFormatter('%d'))
axd['left'].xaxis.set_minor_locator(MultipleLocator(1000))
axd['left'].text(0.0, 1.0, '(a)',transform=axd['left'].transAxes + trans,
            fontsize=20, verticalalignment='top',
            bbox=dict(facecolor='white', edgecolor='none', pad=2))


im2 = axd['right'].contourf(t_plan,levels=np.arange(270,292,2),extend='both',cmap=get_cmap('coolwarm'))
cbar2 = fig.colorbar(im2, orientation="horizontal", cax=axd['cbar2'])
cbar2.ax.tick_params(labelsize=20, pad=6)
cs4 = axd['right'].contour(zlev,colors='k',levels=np.arange(805,1100,10),linewidths=2,alpha=0.8)
axd['right'].clabel(cs4,fmt='%1.0f',inline=1,levels=np.arange(975,1100,20),fontsize=12,colors='black')
axd['right'].set_xlabel('West-East (km)',fontsize=20,labelpad=6)
axd['right'].set_ylabel('South-North (km)',labelpad=6,fontsize=20)
axd['right'].tick_params(axis='both', labelsize=20, which='both', pad=8,length=8)
axd['right'].set_xticks(xax2)
axd['right'].set_xticklabels(['0','','2000','','4000','','6000','',''])
axd['right'].set_yticks(yax2)
axd['right'].set_yticklabels(['0','','2000','','4000','','6000',''])
axd['right'].set_aspect('equal')
axd['right'].text(0.0, 1.0, '(b)',transform=axd['right'].transAxes + trans,
            fontsize=20, verticalalignment='top',
            bbox=dict(facecolor='white', edgecolor='none', pad=2))

plt.savefig('/p/work1/lloveras/adj_4km/figs/t0.pdf',bbox_inches='tight')